## Feature Extaction Test

In [1]:
import sys
sys.path.insert(0, '..')
from metrics.metrics import  quick_circle_comparison, circle_comparison
from general_utils.plots import plot_bboxes_over_image, simple_im_show, simple_im_show2
from metrics.metrics_utils import get_tp_fp_fn_center_patch_criteria
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from mc_candidate_proposal.morphology_mc import MorphologyCalcificationDetection
from feature_extraction.feature_extraction import CandidatesFeatureExtraction, CandidatesFeatureExtraction_MP
from database.dataset import *
import feature_extraction.haar_features.haar_modules as hm

import pickle

### Database initialization

In [2]:
db = INBreast_Dataset(
    return_lesions_mask=True,
    level='image',
    max_lesion_diam_mm=None,
    partitions=['train', 'validation'],
    min_breast_fraction_roi=0.5,
    normalize=None,
    n_jobs=-1,
    use_muscle_mask=True
)

### Detectors Parameters and initialization

In [5]:
hd = MorphologyCalcificationDetection(rbd_img_path='/home/vzalevskyi/projects/calc-det/data/recounstructed_by_dialation_img/new_rbd',
                                      threshold=0.96, min_distance=6,
                                      area=14*14, filter_muscle_region=True, )
CENTER_CROP_PATCH = None
PATCH_SIZE = 14

GABOR_PARAMS = {'scale': 2, 'orientation': 3,
                'max_freq': 0.2, 'ksize': (20, 20), 'sigma': 3}
WAVELET_PARAMS = {'angles': [0, np.pi/4, np.pi/2]}


# THIS IS FINAL HAAR
path = (Path.cwd().parent)
with open(path/'data/final_haar_400_feat_selection_GSM.p', 'rb') as f:
    selection = pickle.load(f)
    
# ISTANTIATE the candidate Feature Extractor with the selected features.
haar_params = {
    'skimage': {
        'feature_type': selection['skimage_haar_feature_types_sel'],
        'feature_coord': selection['skimage_haar_feature_coords_sel']
    },
    'ours': {
        'horizontal_feature_selection': selection['hor_feats_selection'].tolist(),
        'rotated_feature_selection': selection['rot_feats_selection'].tolist(),
        'rotated_feature_types': None,
        'horizontal_feature_types': None
    },
    'patch_size': 14
}

cfe = CandidatesFeatureExtraction_MP(patch_size=PATCH_SIZE, fos=False, gabor_params=None,
                                  wavelet_params=None, haar_params=haar_params)

### Detector initialization

In [6]:
db_range = list(range(len(db)))

### FE

In [8]:
fdf = []
fns_df = []
for idx in tqdm(db_range[:]):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']
    muscle_mask = db_sample['muscle_mask']
    # candidate selection
    candidates = hd.detect(image, image_id, muscle_mask)

    # labeling of candidates:
    tp, fp, fn = get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, CENTER_CROP_PATCH, PATCH_SIZE)
    candidates = pd.concat([tp, fp], axis=0, ignore_index=True)
    
    # Extracting features
    labels = np.where(candidates.label.values == 'TP', True, False)
    
    # Avoid extracting features for repeted detections
    X = candidates.drop_duplicates(subset='repeted_idxs')
    X = cfe.extract_features(X.loc[:,['x','y','radius']].values.astype(int), image)

    # Get features dfs
    X = pd.DataFrame(data=X, columns=cfe.feature_names)
    X.index = candidates.drop_duplicates(subset='repeted_idxs').index

    res = X.loc[candidates.repeted_idxs.tolist(),:]
    res['img_id'] = image_id
    res['repeted_idxs'] = candidates.repeted_idxs.tolist()
    res['matching_gt'] = candidates.matching_gt.tolist()
    res['label'] = labels

    # Generate a fn dataframe to compute frocs
    fn['img_id'] = image_id
    fns_df.append(fn)
    fdf.append(res)
    
data = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)


100%|██████████| 210/210 [17:09<00:00,  4.90s/it]


In [9]:
pd.to_pickle((data, fns_df), 'feature_sets/gmorph_HaarF_final.pkl')

# REST

In [6]:
fdf = []
fns_df = []
for idx in tqdm(range(len(db))):
    # extracting data
    db_sample = db[idx]
    image = db_sample['img']
    image_id = db.df.iloc[idx].img_id
    image_mask = db_sample['lesion_mask']

    # candidate selection
    candidates = hd.detect(image, image_id, load_processed_images=True, hough2=True)[1]

     # labeling of candidates:
    tp, fp, fn = utils.get_tp_fp_fn_center_patch_criteria(
        candidates, image_mask, None, PATCH_SIZE)
    
    candidates = tp + fp
    labels = np.array([True] * len(tp) + [False] * len(fp))
    
    # feature extraction
    X = cfe.extract_features(candidates, image)
    res = pd.DataFrame(X, columns=cfe.feature_names)
    
    res['img_id'] = image_id
    res['labels'] = labels

    
    # Generate a fn dataframe to compute frocs
    fn_df = pd.DataFrame(fn, columns=['x', 'y', 'radius'])
    fn_df['labels'] = 'FN'
    fn_df['img_id'] = image_id
    fns_df.append(fn_df)
    fdf.append(res)

all_data_df = pd.concat(fdf, ignore_index=True)
fns_df = pd.concat(fns_df, ignore_index=True)

100%|██████████| 154/154 [46:18<00:00, 18.04s/it]


In [ ]:
pd.to_pickle((all_data_df, fns_df), 'feature_sets/hough2_all_feat_1pxsize.pkl')

# Test|

In [4]:
cfe = CandidatesFeatureExtraction(patch_size=PATCH_SIZE, fos=True, gabor_params=GABOR_PARAMS,
                                  wavelet_params=WAVELET_PARAMS, haar_params=None)

cfe_mp = CandidatesFeatureExtraction_MP(patch_size=PATCH_SIZE, fos=True, gabor_params=GABOR_PARAMS,
                                  wavelet_params=WAVELET_PARAMS, haar_params=None)


hd = HoughCalcificationDetection()


In [5]:
idx = 0
db_sample = db[idx]
image = db_sample['img']
image_id = db.df.iloc[idx].img_id
image_mask = db_sample['lesion_mask']

# candidate selection
candidates = hd.detect(image, image_id, load_processed_images=True, hough2=True)[1]

    # labeling of candidates:
tp, fp, fn = get_tp_fp_fn_center_patch_criteria(
    candidates, image_mask, None, PATCH_SIZE)

In [6]:
len(candidates)

1776

In [7]:
# feature extraction
X = cfe.extract_features(candidates, image)
res = pd.DataFrame(X, columns=cfe.feature_names)

In [9]:
cfe_mp.n_jobs = 8
# feature extraction
X2 = cfe_mp.extract_features(candidates, image)
res2 = pd.DataFrame(X2, columns=cfe_mp.feature_names)